In [111]:
import pandas as pd
import numpy as np
import json
import random

This function is for choosing a random song in the playlist to find a path between. Alternatively, you can also choose two songs manually.

In [112]:
def random_song(playlists):
    rand_index_playlist = random.randint(0,len(playlists))
    rand_index_track = random.randint(0,len(playlists[rand_index_playlist]['tracks'])-1)
    song = playlists[rand_index_playlist]['tracks'][rand_index_track]['track_uri']
    print(playlists[rand_index_playlist]['tracks'][rand_index_track]['track_name'])

    return song

This function is there to condense the playlist data to only the parameters that are needed.

In [113]:
def new_playlist(name, pid, tracks):
    new_playlist = {}
    new_playlist['name'] = name
    new_playlist['pid'] = pid
    new_playlist['tracks'] = []
    for track in tracks:
        temp_dict = {}
        temp_dict['track_name'] = track['track_name']
        temp_dict['track_uri'] = track['track_uri']
        temp_dict['pos'] = track['pos']
        new_playlist['tracks'].append(temp_dict)

    return new_playlist

The class consists of the following functions:

##### constructor:
set all the parameters to their default values and create lists of all the playlists that contain the starting track and the ending track respectively
##### update_starting_playlists_list:
if no common songs between the current starting playlists and the ending playlists are found, then the starting playlists are changed to the ones that contain the next song in the last starting playlist
##### add_to_path:
add a song and the containing playlist to the path
##### find_common:
recursive function for finding the path between two songs. First, it checks whether the current starting playlists and ending playlist have any songs in common (in the right position, aka after the starting song and before the ending song). If not, then it sets the starting playlists to the ones containing the next song in the first starting playlists and calls the function again with the new starting playlists. When a result is found, the path is added and the function returns.
##### print_path:
adds the beginning song to the playlists, adds the jumping songs to the paths and reverses the path, since the list begins with the last result, aka the ending song. Then, it prints the full path in a formatted way

In [114]:
class PathFinder():
    
    playlists = {}
    complete = False
    checked_playlists = []
    start_song_uri = ''
    end_song_uri = ''
    starting_playlists = []
    end_playlists = []
    path = []

    def __init__(self, playlists, start_song_uri, end_song_uri):

        self.playlists = playlists
        self.start_song_uri = start_song_uri
        self.end_song_uri = end_song_uri
        self.path = []
        self.checked_playlists = []
        self.starting_playlists = []
        self.end_playlists = []
        self.complete = False

        for playlist in self.playlists:
            for track in playlist['tracks']:
                if start_song_uri == track['track_uri']:
                    self.starting_playlists.append(new_playlist(playlist['name'], playlist['pid'], playlist['tracks'][track['pos']:]))
                if end_song_uri == track['track_uri']:
                    self.end_playlists.append(new_playlist(playlist['name'], playlist['pid'], playlist['tracks'][:track['pos']+1]))

    def update_starting_playlists_list(self, song):
        self.starting_playlists = []
        for playlist in self.playlists:
            if playlist['pid'] not in self.checked_playlists:
                for track in playlist['tracks']:
                    if song['track_uri'] == track['track_uri']:
                        self.starting_playlists.append(new_playlist(playlist['name'], playlist['pid'], playlist['tracks'][track['pos']:]))

    def add_to_path(self, song, playlist):
        temp_dict = {}
        temp_dict['song'] = song
        temp_dict['playlist'] = playlist['pid']
        temp_dict['playlist_name'] = playlist['name']
        self.path.append(temp_dict)

    def find_common(self):
        for start_playlist in self.starting_playlists:
            for track_a in start_playlist['tracks']:
                for end_playlist in self.end_playlists:
                    for track_b in end_playlist['tracks']:
                        
                        if(track_b['track_uri'] == track_a['track_uri']):
                            
                            self.complete = True

                            for track in end_playlist['tracks']:
                                if self.end_song_uri == track['track_uri']:
                                    self.add_to_path(track, end_playlist)
                                    break
                                
                            self.add_to_path(track_a, start_playlist)
                            return

            self.checked_playlists.append(start_playlist['pid'])

        for playlist in self.starting_playlists:
            for track in playlist['tracks']:

                if not self.complete:  
                    self.update_starting_playlists_list(track)
                    self.find_common()
                    if self.complete:
                        self.add_to_path(track, playlist)
                        return 
                else:
                    return 
    
    def print_path(self):
        if len(self.path) != 0:
            goal_playlist = self.playlists[int(self.path[-1]['playlist'])]
            for track in goal_playlist['tracks']:
                if self.start_song_uri == track['track_uri']:
                    self.add_to_path(track, goal_playlist)

            path_without_jumpers = self.path
            self.path = []

            for i, p in enumerate(path_without_jumpers):
                if i != 0:
                    current_song = p['song']['track_uri']
                    goal_playlist = self.playlists[int(path_without_jumpers[i-1]['playlist'])]
                    self.add_to_path(path_without_jumpers[i-1]['song'], goal_playlist)
                    for track in goal_playlist['tracks']: 
                        if current_song == track['track_uri']:
                            self.add_to_path(track, goal_playlist)
                            break

            new_path = self.path.copy()
            new_path.reverse()
            self.path = new_path

        for i, p in enumerate(self.path):
            if i == 0:
                print('Starting Track: "', p['song']['track_name'], '" in Playlist "', p['playlist_name'], '" at Position', p['song']['pos'])
            elif i == len(self.path)-1:
                print('Ending Track: "', p['song']['track_name'], '" in Playlist "', p['playlist_name'], '" at Position', p['song']['pos'])
            else:
                print('"', p['song']['track_name'], '" in Playlist "', p['playlist_name'], '" at Position', p['song']['pos'])

Load the playlists data

In [115]:

file = open('mpd.slice.0-999.json')
data = json.load(file)
playlists = data['playlists']

Create objects with two songs that you want to find a path inbetween. Use the random_song function to choose a random song out of the playlists.

In [116]:
# call PathFinder with determined songs
obj = PathFinder(playlists, 'spotify:track:6Umac95Mt46VcwAM9s9mOa', 'spotify:track:0qg7vnrsBfsDCikGxyWzSX')
obj.find_common()
obj.print_path()

print('\n')

# call PathFinder with random songs
obj2 = PathFinder(playlists, random_song(playlists), random_song(playlists))
obj2.find_common()
obj2.print_path()

Starting Track: " To the Moon " in Playlist " 😴😴 " at Position 6
" LOVE. FEAT. ZACARI. " in Playlist " 😴😴 " at Position 7
" LOVE. FEAT. ZACARI. " in Playlist " xx " at Position 79
" Location " in Playlist " xx " at Position 80
" Location " in Playlist " in my feels " at Position 54
" Adorn " in Playlist " in my feels " at Position 199
" Adorn " in Playlist " VIBE " at Position 16
Ending Track: " Organ Donor " in Playlist " VIBE " at Position 66


Tomorrow Will Be Better
This Is Real
Starting Track: " Tomorrow Will Be Better " in Playlist " Reggae " at Position 24
" The Drugs " in Playlist " Reggae " at Position 33
" The Drugs " in Playlist " study " at Position 47
Ending Track: " This Is Real " in Playlist " study " at Position 127
